### 데이터 다운(미완성)

In [ ]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome('./chromedriver.exe')

# month = 3
date_range = (pd.date_range(start='20200301', end='20200331')).strftime("%Y-%m-%d").tolist()

driver.get('https://www.bigkinds.or.kr/')
# 공지 팝업창 삭제
driver.find_element_by_xpath('//*[@id="popup-dialog-54"]/div[2]/div/div[1]/label/input').click()
# 로그인
driver.find_element_by_xpath('//*[@id="app-top-navbar"]/ul/li[1]/a').click()
driver.find_element_by_xpath('//*[@id="login-modal"]/div/div/div[2]/div/div[3]/a[3]').send_keys(Keys.ENTER)
driver.find_element_by_xpath('//input[@type="email"]').send_keys('leejm456@gmail.com')
driver.find_element_by_xpath('//*[@id="identifierNext"]/div/button/div[2]').click()

WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//input[@type="password"]'))).send_keys("ljm6!!8@@")
driver.find_element_by_xpath('//input[@type="password"]').send_keys(Keys.ENTER)
driver.implicitly_wait(20) 

for date in date_range[:1]:
    # 기간 입력 창 열기
    driver.find_element_by_xpath('//*[@id="date-filter-btn"]').click()
    # 기간 입력 (왜 안지워질까?)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-begin-date"]'))).clear()
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-begin-date"]'))).send_keys(date)

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-end-date"]'))).clear()
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-end-date"]'))).send_keys(date)
    # 적용 버튼 누르기
    driver.find_element_by_xpath('//*[@id="date-confirm-btn"]').click()
    driver.find_element_by_xpath('//*[@id="news-search-form"]/div/div/div/div[1]/span/button').click()

    # 분석결과 누르기
    driver.find_element_by_xpath('//*[@id="headingThree"]/h4/a').click()
    driver.find_element_by_xpath('//*[@id="analytics-data-download"]/div/div[2]/button').click()
    driver.switch_to.alert.accept()
driver.quit()

### 워드클라우드

In [1]:
import os, re
import pandas as pd
from soynlp.tokenizer import LTokenizer
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt

def remove_stopword(title_ls):

    stopword = '[^가-힣0-9\s]'

    return [re.sub(stopword, '', str(sents)) for sents in title_ls]

def tokenizer(sent_ls):

    token_ls = []
    tokenizer = LTokenizer()
    
    for sent in sent_ls:
        token_ls.extend(tokenizer.tokenize(sent))
    
    return token_ls

def draw_wordcloud(token_ls):

    count = Counter(token_ls)
    words = dict(count.most_common(100))
    
    wordcloud = WordCloud(font_path = 'C:/Windows/Fonts/malgun.ttf', 
                    background_color='white',
                    colormap = "rainbow_r").generate_from_frequencies(words) 
    plt.figure(figsize=(10, 6))
    plt.title(os.path.splitext(file)[0].split('-')[-1], fontsize=15)
    plt.imshow(wordcloud) 
    plt.axis('off') 
    plt.savefig('./bigkinds_wc/'+os.path.splitext(file)[0].split('-')[-1]+'_wordcloud.jpg')

In [ ]:
path = './bigkinds/'
file_list = os.listdir(path)

for file in file_list:
    excel = pd.read_excel(path+file)
    
    # 불용어 제거 
    title_ls = excel['제목'].tolist()
    sent_ls = remove_stopword(title_ls)

    # 토큰화
    token_ls = tokenizer(sent_ls)

    # 필요없는 단어 삭제
    token_ls = [token for token in token_ls  if token != '코로나']
    token_ls = [token for token in token_ls  if token != '코로나19']
    token_ls = [token for token in token_ls  if token != '신종코로나']

    # 워드클라우드
    draw_wordcloud(token_ls)